# Practica 4
## Captcha Destroyer :por: Python Eaters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán

## Import


In [ ]:
import string

import os
import numpy as np 
import cv2 #OpenCV

## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos, para esto usaremos cv2 (OpenCV)

Primero necesitamos saber cuales son los caracteres que nos podemos encontrar en los 'catcha' y, ya que estamos, la cantidad de caracteres que son.

In [ ]:
#total no of images in dataset
direccion_examples = "C:\Users\NITROPC\OneDrive\Escritorio\Daniel\Trabajo\Universidad\4º Año\2ºCuatri\SI\Practica 4\SI_CaptchaDestroller\samples"
n = len(os.listdir(direccion_examples))
n


symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
numChar = len(symbols) # Total number of char possible
numChar

Despues queremos preprocesar las imagenes con las que estemos trabajando, para asegurarnos que todas tengan el mismo formato y dimensiones.

In [ ]:
def preprocesar_img():
    img_array = np.zeros((n,50,200,1)) #1070*50*200 array with all entries 0
    letersImg_array = np.zeros((5,n,numChar)) #5*1070*36(5 letters in captcha) with all entries 0

    

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual. (logic CNN)

## Parte 2: Reconocer una cadena de dígitos y letras bien separadas
Ahora que sabemos identificar letras y números por separado podemos agruparlos bien separados en una cadena para leerlos y escribirlos en orden de aparición. (logic RNN)

## Parte 3: Reconocer dígitos y letras que se juntan/mezclan
Por último queda ser capaces de distinguir los números y letras que están solapados en la cadena para poder escribirlos (CRNN = CNN + RNN)